In [1]:
#Importing Libraries
import pandas as pd
import nltk
from textblob import TextBlob
from nltk.corpus import stopwords
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import scale
import numpy as np

In [2]:
#Read the twitter dataset from the file
data = pd.read_csv('dataset.csv',index_col=False )
print(data.shape)
data.head(6)


(99996, 2)


,tweet,label
0,Beats by Dr. Dre urBeats Wired In-Ear Headphon...,2
1,RT @Papapishu: Man it would fucking rule if we...,3
2,It is time to draw close to Him &#128591;&#127...,1
3,if you notice me start to act different or dis...,1
4,"Forget unfollowers, I believe in growing. 7 ne...",1
5,RT @Vitiligoprince: Hate Being sexually Frustr...,3


In [3]:
# Splitting into train and test
X_train, X_test, y_train, y_test = train_test_split(data.tweet,data.label,test_size = 0.2)

In [4]:
# Clean the text by removing ...

def to_lower(word):
    result = word.lower()
    return result

def remove_hyperlink(word):
    return  re.sub(r"http\S+", "", word)

def remove_mentions(word):
    return re.sub(r"@\S+", "", word)

def remove_number(word):
    result = re.sub(r'\d+', '', word)
    return result

def remove_punctuation(word):
    result = re.sub('[^A-Za-z]+', ' ', word)
    return result

def remove_whitespace(word):
    result = word.strip()
    return result

def replace_newline(word):
    return word.replace('\n','')

def remove_stopwords(word):
    stopwords = nltk.corpus.stopwords.words('english')
    return ' '.join(word for word in i.split() if word not in stopwords)

def clean_up_pipeline(sentence):
    cleaning_data = [remove_hyperlink,
                      replace_newline,
                      to_lower,
                      remove_number,
                      remove_punctuation,
                      remove_whitespace]
    for func in cleaning_data:
        
        sentence = func(sentence)
    return sentence


X_train = X_train.apply(clean_up_pipeline)
X_test = X_test.apply(clean_up_pipeline)
print(X_train[:3])
print(X_train.shape)
print(X_test[:3])
print(X_test.shape)

28302    so this is why your curly haircut costs that m...
46462    seancooper but who at at dt mcdowell damn near...
56688    bosslogic then teach me how to handle those ha...
Name: tweet, dtype: object
(79996,)
1089     leadership is not just what happens when you r...
80412    even satan will bow it says every why wait pra...
65358    have a wonderful weekend whatever you re up to...
Name: tweet, dtype: object
(20000,)


# Tokenization

In [5]:

# X_train = X_train.apply(word_tokenize)
# X_test = X_test.apply(word_tokenize)
print(X_train[:3])
print(X_train.shape)
print(X_test[:3])
print(X_test.shape)

28302    so this is why your curly haircut costs that m...
46462    seancooper but who at at dt mcdowell damn near...
56688    bosslogic then teach me how to handle those ha...
Name: tweet, dtype: object
(79996,)
1089     leadership is not just what happens when you r...
80412    even satan will bow it says every why wait pra...
65358    have a wonderful weekend whatever you re up to...
Name: tweet, dtype: object
(20000,)


# Stemming

In [6]:

stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text])
X_train = X_train.apply(lambda text: stem_words(text))
X_test = X_test.apply(lambda text: stem_words(text))

In [7]:
# from gensim.models.word2vec import Word2Vec 
# #Word Embedding using Word2Vec
# wordvector = Word2Vec(vector_size=300, min_count=10)
# wordvector.build_vocab([x for x in X_train])
# wordvector.train([x for x in X_train],total_examples=wordvector.corpus_count, epochs=1)

In [8]:
# wordvector.wv['a']

In [9]:
# from gensim.models import KeyedVectors
# # Load vectors directly from the file
# model = KeyedVectors.load_word2vec_format('data/GoogleGoogleNews-vectors-negative300.bin', binary=True)
# # Access vectors for specific words with a keyed lookup:
# vector = model['easy']
# # see the shape of the vector (300,)
# vector.shape
# # Processing sentences is not as simple as with Spacy:
# vectors = [model[x] for x in "This is some text I am processing with Spacy".split(' ')]

In [19]:
import gensim.downloader as api
# path = api.load("word2vec-google-news-300", return_path=True)
# print(path)

In [20]:
import gensim
glove_vectors = gensim.downloader.load('word2vec-google-news-300')

In [16]:
# glove_vectors['a']

In [21]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += glove_vectors[word].reshape((1, size)) #* tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [22]:
from tqdm import tqdm
import numpy as np
tqdm.pandas(desc="progress-bar")
train_vecs_w2v = np.concatenate([buildWordVector(z, 300) for z in tqdm(map(lambda x: x, X_train))])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, 300) for z in map(lambda x: x, X_test)])
test_vecs_w2v = scale(test_vecs_w2v)



0it [00:00, ?it/s]
181it [00:00, 1802.20it/s]
372it [00:00, 1861.65it/s]
566it [00:00, 1895.84it/s]
771it [00:00, 1954.62it/s]
979it [00:00, 1998.01it/s]
1179it [00:00, 1681.97it/s]
1378it [00:00, 1770.92it/s]
1565it [00:00, 1798.32it/s]
1750it [00:00, 1808.52it/s]
1949it [00:01, 1859.28it/s]
2155it [00:01, 1916.58it/s]
2355it [00:01, 1941.06it/s]
2556it [00:01, 1961.58it/s]
2762it [00:01, 1990.44it/s]
2962it [00:01, 1975.37it/s]
3166it [00:01, 1992.60it/s]
3366it [00:01, 1991.39it/s]
3567it [00:01, 1994.58it/s]
3767it [00:01, 1985.19it/s]
3966it [00:02, 1934.92it/s]
4170it [00:02, 1964.39it/s]
4373it [00:02, 1982.04it/s]
4572it [00:02, 1967.29it/s]
4769it [00:02, 1963.09it/s]
4966it [00:02, 1957.00it/s]
5162it [00:02, 1950.91it/s]
5359it [00:02, 1953.69it/s]
5557it [00:02, 1960.17it/s]
5756it [00:02, 1967.28it/s]
5953it [00:03, 1954.32it/s]
6149it [00:03, 1939.69it/s]
6347it [00:03, 1951.20it/s]
6545it [00:03, 1956.88it/s]
6747it [00:03, 1974.38it/s]
6945it [00:03, 1963.08it/s]
7142i

In [23]:
print(y_train)
from keras.utils import to_categorical
encoded = to_categorical(y_train)
encoded

28302    1
46462    1
56688    3
56418    1
2229     1
        ..
42798    1
4762     1
21434    1
52953    1
6960     1
Name: label, Length: 79996, dtype: int64


0it [11:54, ?it/s]


array([[0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)

In [ ]:
from keras.models import Sequential
from keras.models import Model
from keras.layers import *
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=300))
model.add(Dense(5, activation='softmax'))
# adam=keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer="adam",
              loss='CategoricalCrossentropy',
              metrics=['accuracy'])

model.fit(train_vecs_w2v, encoded, epochs=300, batch_size=32, verbose=2)

Epoch 1/300
2500/2500 - 5s - loss: 0.9541 - accuracy: 0.6403
Epoch 2/300
2500/2500 - 5s - loss: 0.9122 - accuracy: 0.6551
Epoch 3/300
2500/2500 - 5s - loss: 0.9019 - accuracy: 0.6568
Epoch 4/300
2500/2500 - 5s - loss: 0.8977 - accuracy: 0.6589
Epoch 5/300
2500/2500 - 5s - loss: 0.8942 - accuracy: 0.6605
Epoch 6/300
2500/2500 - 5s - loss: 0.8905 - accuracy: 0.6605
Epoch 7/300
2500/2500 - 5s - loss: 0.8881 - accuracy: 0.6620
Epoch 8/300
2500/2500 - 5s - loss: 0.8863 - accuracy: 0.6624
Epoch 9/300
2500/2500 - 5s - loss: 0.8839 - accuracy: 0.6634
Epoch 10/300
2500/2500 - 5s - loss: 0.8824 - accuracy: 0.6643
Epoch 11/300
2500/2500 - 5s - loss: 0.8808 - accuracy: 0.6648
Epoch 12/300
2500/2500 - 5s - loss: 0.8800 - accuracy: 0.6658
Epoch 13/300
2500/2500 - 5s - loss: 0.8792 - accuracy: 0.6666
Epoch 14/300
2500/2500 - 5s - loss: 0.8779 - accuracy: 0.6663
Epoch 15/300
2500/2500 - 5s - loss: 0.8773 - accuracy: 0.6669
Epoch 16/300
2500/2500 - 5s - loss: 0.8761 - accuracy: 0.6660
Epoch 17/300
2500

In [ ]:
encoded_y_test = to_categorical(y_test)
encoded_y_test
score = model.evaluate(test_vecs_w2v, encoded_y_test, verbose=2)
print(score[1])

In [ ]:
from lightgbm import LGBMClassifier
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

# params={}
# params['learning_rate']=0.03
# params['boosting_type']='gbdt' #GradientBoostingDecisionTree
# params['objective']='multiclass' #Multi-class target feature
# params['metric']='multi_logloss' #metric for multi-class
# params['max_depth']=10
# params['num_class']=5 #no.of unique values in the target class not inclusive of the end value

# OLGBM =LGBMClassifier(**params)
SLGBM =LGBMClassifier(n_estimators =1000,  n_jobs=-1,verbose=1)
clf = make_pipeline(sc,SLGBM)
clf.fit(train_vecs_w2v, y_train)


In [ ]:
def Predict_Result(y, predicted,predicted_proba):
   

	
	confusion = confusion_matrix(y, predicted)

	TP = confusion[1, 1]
	TN = confusion[0, 0]
	FP = confusion[0, 1]
	FN = confusion[1, 0]


	# Specificity
	SPE_cla = (TN/float(TN+FP))*100 

	# False Positive Rate
	FPR = (FP/float(TN+FP))

	#False Negative Rate (Miss Rate)
	FNR = (FN/float(FN+TP))

	#Balanced Accuracy
	ACC_Bal = 0.5*((TP/float(TP+FN))+(TN/float(TN+FP)))*100

	# compute MCC
	MCC_cla = matthews_corrcoef(y, predicted)
	F1_cla = f1_score(y, predicted)
	PREC_cla = precision_score(y, predicted)*100
	REC_cla = recall_score(y, predicted)*100
	Accuracy_cla = accuracy_score(y, predicted)*100


	ROC_auc_score= roc_auc_score(y, predicted_proba)*100
	PR_auc_score= average_precision_score(y, predicted_proba)*100
	sw = (REC_cla+SPE_cla-1)



	print(f"SN (%), {REC_cla:.2f}")
	print(f"SP (%), {SPE_cla:.2f}")
	print(f"Sw (%), {sw:.2f}")
	print(f"BACC (%), {ACC_Bal:.2f}")
	print(f"MCC, {MCC_cla:.3f}")
	print(f"ACC (%), {Accuracy_cla:.2f}") 
	print(f"FPR, {FPR:.3f}")
	print(f"FNR, {FNR:.3f}")
	print(f"PR (%), {PREC_cla:.2f}")
	print(f"F1-score, {F1_cla:.3f}") 
	print(f"ROCAUC (%), {ROC_auc_score:.2f}")
	print(f"PRAUC (%), {PR_auc_score:.2f}")







In [ ]:
from sklearn.metrics import *
predicted=clf.predict(test_vecs_w2v)
predicted_proba=clf.predict_proba(test_vecs_w2v)
# threshold=0.5
# y_pred = (prob_predict0[:,1] >= threshold).astype(int)
# Predict_Result(y_test.astype(float), np.array(y_pred),prob_predict0[:,1])

In [ ]:
print(prob_predict0)
y=y_test
MCC_cla = matthews_corrcoef(y, predicted)
F1_cla = f1_score(y, predicted,average="micro")
PREC_cla = precision_score(y, predicted,average="micro")*100
REC_cla = recall_score(y, predicted,average="micro")*100
Accuracy_cla = accuracy_score(y, predicted)*100
ROC_auc_score= roc_auc_score(y, predicted_proba,multi_class="ovr")*100

print(f"SN (%), {REC_cla:.2f}")
print(f"MCC, {MCC_cla:.3f}")
print(f"ACC (%), {Accuracy_cla:.2f}") 
print(f"PR (%), {PREC_cla:.2f}")
print(f"F1-score, {F1_cla:.3f}") 
print(f"ROCAUC (%), {ROC_auc_score:.2f}")
